#### Objective: To train a custom dataset in YOLOv8 predicting two classes(awake and drowsy)

In [2]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...


##### Importing required libraries

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import cv2
from ultralytics import YOLO
import os
import time

* Uuid is used to create unique identifier

In [3]:
import uuid

Setting up the path

In [27]:
IMAGES_PATH = os.path.join('Data', 'images')
labels = ['awake', 'drowsy']
number_imgs =20

##### Making our own dataset

In [29]:
cap = cv2.VideoCapture(0)

#Loop through labels
for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)

    #Loop through images
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
        #Webcam feed
        ret, frame = cap.read()
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        # Write out image to file
        cv2.imwrite(imgname, frame)

        # Render to the screen
        cv2.imshow('Image Collection', frame)
        
        # 2 second delay between captures
        time.sleep(2)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

### Custom labeing our dataset

##### Labeling using Tzutalin LabelImg

Labeling is done on LabelImg GUI. It is important that saved labels should be set to YOLO format i.e., (class_index, x_center, y_center, height, width)

In [30]:
!git clone https://github.com/heartexlabs/labelImg

Cloning into 'labelImg'...


In [31]:
!pip install pyqt5 lxml --upgrade


                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
                                              0.0/6.8 MB ? eta -:--:--
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -                                        0.2/6.8 MB 696.3 kB/s eta 0:00:10
     -        

Unable to open libx/resources.py for writing


In [32]:
!cd labelImg && pyrcc5 -o libs/resources.py resources.qrc

#### Training our custom model

* Epoch is set to 500. However, training will end at epoch 165 as the default patience is set to be 50 and no further improvements happened.
* Image size is given as 320.

In [9]:
!yolo detect train data=dataset.yaml model=yolov8n.pt epochs=500 imgsz=320

Ultralytics YOLOv8.0.132  Python-3.11.3 torch-2.0.1+cu117 CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset.yaml, epochs=500, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, wo

##### Inference on our custom dataset

In [5]:
# Trained weights from custom dataset
model = YOLO('runs/detect/train9/weights/last.pt')  

results = model('Data/images/awake.005e5857-1fe6-11ee-9a54-a0806977dfcf.jpg') 


image 1/1 c:\Users\lm60008\my-env\Data\images\awake.005e5857-1fe6-11ee-9a54-a0806977dfcf.jpg: 256x320 1 awake, 83.3ms
Speed: 4.0ms preprocess, 83.3ms inference, 6.0ms postprocess per image at shape (1, 3, 256, 320)


In the above inference, we can see it is correcly predicted the class as "awake".

##### Running inference on real time using WebCam

In [6]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    #Make Predictions
    results = model(frame)

    #Output & Predictions
    cv2.imshow('YOLO', results[0].plot())
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


0: 256x320 1 awake, 48.2ms
Speed: 3.0ms preprocess, 48.2ms inference, 3.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 58.2ms
Speed: 1.2ms preprocess, 58.2ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 53.1ms
Speed: 1.0ms preprocess, 53.1ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 54.1ms
Speed: 1.0ms preprocess, 54.1ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 47.4ms
Speed: 1.0ms preprocess, 47.4ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 51.0ms
Speed: 1.0ms preprocess, 51.0ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 50.7ms
Speed: 1.0ms preprocess, 50.7ms inference, 1.0ms postprocess per image at shape (1, 3, 256, 320)

0: 256x320 1 awake, 43.4ms
Speed: 2.0ms preprocess, 43.4ms inference, 2.0ms postprocess per image at shape (1, 3, 256, 320)


##### Key Observations:

* As the custom dataset contains only 40 images with 'opening or closing eye' as a variation between both the classes, the model seems to predict the face as "drowsy" even someone squints or covers their eyes.
* However, aside from the above caveat, our model performed with high confidence rate (>80 percent for most of the cases).
* The model can be improved with more data and different variations which captures the essence of being awake and drowsy.